In [1]:
import os
import sys
import time

import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
from torch.nn.utils.rnn import pad_sequence
from torch.utils.data import Dataset, DataLoader, Subset

import nevergrad as ng

module_path = os.path.abspath(os.path.join('..'))
if module_path not in sys.path:
    sys.path.append(module_path)
    
from python.models import TCN
from python.data import Data, collate_fn
from python.optimizations import (weight_norm, VariationalDropout, 
    VariationalHidDropout, WeightDrop, embedded_dropout)

from python.utils import *

In [2]:
batch_size = 1 

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

print('Torch version: {}'.format(torch.__version__))
print('Device: %s' % (device))
if torch.cuda.device_count() > 1: print("Let's use", torch.cuda.device_count(), "GPUs!")

Torch version: 1.0.0
Device: cpu


In [3]:
data = pd.read_csv('../data/data.csv', index_col=0)

In [4]:
np.random.seed(1111)

train_valid_ratio = 0.9

train_indices = np.sort(np.random.choice(data['idx'], int(len(data)*train_valid_ratio), replace=False))
valid_indices = np.sort(np.array(list(set(np.arange(len(data))) - set(train_indices))))

train_set = Data(data.iloc[train_indices])
valid_set = Data(data.iloc[valid_indices])

train_loader = DataLoader(dataset=train_set, batch_size=batch_size, shuffle=True, 
                         drop_last=True, collate_fn=collate_fn)
valid_loader = DataLoader(dataset=valid_set, batch_size=len(valid_set), collate_fn=collate_fn)

In [8]:
clip = 0.2



In [16]:
# dummy function
def train(nhid, levels, kernel_size, dropout, epochs=3):
    
    print('call train()')
    
    input_size = 120
    output_size = 2
    
    clip = 0.2
    
    model = TCN(input_size, output_size, nhid, levels, kernel_size, dropout).to(device)

    if torch.cuda.device_count() > 1:
        model = nn.DataParallel(model)
        
    optimizer = optim.Adam(model.parameters(), lr=5e-5, betas=(0.9, 0.999), eps=1e-08)
    
    loss_function = nn.NLLLoss(weight=torch.tensor([1., 70.], dtype=torch.double).to(device))

        
    for epoch in range(epochs):  
        for i, (feature, label) in enumerate(train_loader):

            # Clear out accumulates gradients 
            model.zero_grad()

            # Forward pass
            out = model(feature.to(device))

            # Backward propagation
            loss = loss_function(out, label.to(device))
#             train_loss_vec.append(loss.item())
            loss.backward()    
            if clip > 0:
                torch.nn.utils.clip_grad_norm_(model.parameters(), clip)
            optimizer.step()

        with torch.no_grad():
            input, label = next(iter(valid_loader))

            out = model(input.to(device))
            loss = loss_function(out, label.to(device))
#             valid_loss_vec.append(loss.item())

            print('Epoch: {:2d}   Validation Loss: {:.3e}'.format(epoch+1, loss.item()))

    #     torch.save(model.state_dict(), '../models/lstm_ballroom_epoch'+ 
    #                str(epoch+1)+'_vloss_{:.5}.pt'.format(loss.item()))

    return loss

In [18]:
# argument transformation

nhid = ng.var.OrderedDiscrete([60, 80, 100, 120])
levels =  ng.var.OrderedDiscrete([6, 7, 8])
kernel_size = ng.var.OrderedDiscrete([4, 5, 6])
dropout = ng.var.OrderedDiscrete([0.1, 0.2, 0.3, 0.4, 0.5])

# nhid = ng.var.Scalar(int).bounded(10, 200)
# dropout = ng.var.Scalar(float).bounded(0.1, 0.5)

budget = 10 

instrum = ng.Instrumentation(nhid, levels, kernel_size, dropout)

optimizer = ng.optimizers.OnePlusOne(instrumentation=instrum, budget=budget)

recommendation = optimizer.optimize(train)

In [74]:
print(recommendation)

Candidate(args=(55, 6, 7, 0.1999999756279673), kwargs={})
